In [17]:
!pip install -q mediapipe opencv-python tensorflow numpy pandas scikit-learn

In [18]:
import mediapipe as mp
import cv2
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import pickle

In [19]:
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=1
)
detector = vision.HandLandmarker.create_from_options(options)
print("Detector ready")

Detector ready


In [20]:
def extract_landmarks(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)

    result = detector.detect(mp_image)

    if not result.hand_landmarks:
        return None

    hand = result.hand_landmarks[0]
    vector = []
    for lm in hand:
        vector.extend([lm.x, lm.y, lm.z])

    return vector

In [21]:
DATASET_PATH = "/content/isl_dataset/Indian"
MAX_IMAGES_PER_CLASS = 100

X = []
y = []

folders = sorted(os.listdir(DATASET_PATH))
print(f"Found {len(folders)} classes: {folders}\n")

for label in folders:
    label_path = os.path.join(DATASET_PATH, label)

    if not os.path.isdir(label_path):
        continue

    image_files = [f for f in os.listdir(label_path)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    image_files = image_files[:MAX_IMAGES_PER_CLASS]

    success_count = 0

    for img_file in image_files:
        img_path = os.path.join(label_path, img_file)
        features = extract_landmarks(img_path)

        if features is not None:
            X.append(features)
            y.append(label)
            success_count += 1

    print(f"{label}: {success_count}/{len(image_files)} images processed")

X = np.array(X)
y = np.array(y)

print(f"\nFinal dataset: X shape = {X.shape}, y shape = {y.shape}")
print(f"Unique classes: {len(np.unique(y))}")

Found 35 classes: ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

1: 100/100 images processed
2: 100/100 images processed
3: 100/100 images processed
4: 100/100 images processed
5: 100/100 images processed
6: 100/100 images processed
7: 100/100 images processed
8: 100/100 images processed
9: 100/100 images processed
A: 100/100 images processed
B: 100/100 images processed
C: 93/100 images processed
D: 100/100 images processed
E: 100/100 images processed
F: 100/100 images processed
G: 100/100 images processed
H: 100/100 images processed
I: 100/100 images processed
J: 100/100 images processed
K: 100/100 images processed
L: 100/100 images processed
M: 100/100 images processed
N: 83/100 images processed
O: 95/100 images processed
P: 85/100 images processed
Q: 66/100 images processed
R: 100/100 images processed
S: 51/100 images processed
T: 100/100 images processed

In [23]:
if len(X) == 0:
    print("ERROR: No data loaded")
else:
    print(f"SUCCESS: {len(X)} samples loaded")
    print(f"Feature size: {X.shape[1]}")
    print(f"Classes: {sorted(np.unique(y))}")

SUCCESS: 3373 samples loaded
Feature size: 63
Classes: [np.str_('1'), np.str_('2'), np.str_('3'), np.str_('4'), np.str_('5'), np.str_('6'), np.str_('7'), np.str_('8'), np.str_('9'), np.str_('A'), np.str_('B'), np.str_('C'), np.str_('D'), np.str_('E'), np.str_('F'), np.str_('G'), np.str_('H'), np.str_('I'), np.str_('J'), np.str_('K'), np.str_('L'), np.str_('M'), np.str_('N'), np.str_('O'), np.str_('P'), np.str_('Q'), np.str_('R'), np.str_('S'), np.str_('T'), np.str_('U'), np.str_('V'), np.str_('W'), np.str_('X'), np.str_('Y'), np.str_('Z')]


In [24]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")

Train: (2698, 63)
Test: (675, 63)


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

num_classes = len(np.unique(y_encoded))

model = Sequential([
    Dense(256, activation='relu', input_shape=(63,)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 35)             │         2,275 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,347 (239.64 KB)

 Trainable params: 60,579 (236.64 KB)

 Non-trainable params: 768 (3.00 KB)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=60,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9628 - loss: 0.1180 - val_accuracy: 0.9985 - val_loss: 0.0027 - learning_rate: 1.0000e-05
Epoch 2/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9665 - loss: 0.1025 - val_accuracy: 0.9985 - val_loss: 0.0027 - learning_rate: 1.0000e-05
Epoch 3/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9702 - loss: 0.0913 - val_accuracy: 0.9985 - val_loss: 0.0028 - learning_rate: 1.0000e-05
Epoch 4/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9705 - loss: 0.1134 - val_accuracy: 0.9985 - val_loss: 0.0028 - learning_rate: 1.0000e-05
Epoch 5/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9683 - loss: 0.1051 - val_accuracy: 0.9985 - val_loss: 0.0027 - learning_rate: 1.0000e-05
Epoch 6/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9610 - loss: 0.1200 - val_accuracy: 0.9985 - val_loss: 0.0026 - learning_rate: 1.0000e-05
Epoch 7/60
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9695 - loss:

In [30]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"Train Accuracy: {train_acc * 100:.2f}%")


Test Accuracy: 99.85%
Test Loss: 0.0024
Train Accuracy: 99.93%


In [31]:
model.save("isl_hand_model.h5")
print("Model saved")

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)
print("Encoder saved")

Model saved
Encoder saved


In [32]:
sample_idx = np.random.randint(0, len(X_test))
sample = X_test[sample_idx].reshape(1, -1)
actual = encoder.inverse_transform([y_test[sample_idx]])[0]

pred = model.predict(sample, verbose=0)
predicted = encoder.inverse_transform([np.argmax(pred)])[0]

print(f"Actual: {actual}")
print(f"Predicted: {predicted}")
print(f"Confidence: {np.max(pred) * 100:.2f}%")

Actual: E
Predicted: E
Confidence: 99.84%
